In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [12]:
!mkdir mixtral-simple-math
# !rm mixtral-factual-wrong-v2/*.json

In [13]:
# prompt = 'generate more factual wrong questions ONLY, like, why kangaroo in malaysia, can expand beyond animals'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text
# output

In [14]:
# prompt = 'generate more math questions ONLY'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text
# output

In [10]:
prompt = 'generate more simple math questions ONLY, mathematical symbol and numbers ONLY'
formatted_prompt = format_prompt(prompt, [])
stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
output = stream.generated_text
output

' 1. What is 2 + 2?\n2. What is 5 - 3?\n3. What is 4 \\* 3?\n4. What is 9 / 3?\n5. What is 6 \\* (4 + 2)?\n6. What is 10 - (3 + 2)?\n7. What is 25 / 5?\n8. What is 8 \\* 8 \\* 8?\n9. What is 100 - 50?\n10. What is 12 + 12 + 12?\n11. What is 20 / 2 / 2?\n12. What is 10 + 10 \\* 10?\n13. What is 81 - 27?\n14. What is 3 \\* 3 \\* 3 \\* 3?\n15. What is 1000 / 10?\n16. What is 10 \\* 10 \\* 10 \\* 10?\n17. What is 16 - 9?\n18. What is 5 \\* 5 \\* 5?\n19. What is 20 / 5 / 2?\n20. What is 10 + 10 \\* 2?\n\nPlease let me know if you have any question or if you need me to generate more.'

In [15]:
def answer(q, i):
    filename = f'mixtral-simple-math/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [37]:
prompts = ['generate more simple math questions ONLY, mathematical symbol and numbers ONLY'] * 10000

In [38]:
len(prompts)

10000

In [39]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|█████████████████████████████████████████| 200/200 [48:06<00:00, 14.43s/it]


In [49]:
from glob import glob

files = glob('mixtral-simple-math/*.json')
files = sorted(files, key = lambda x: int(x.split('/')[-1].replace('.json', '')))
        
len(files)

5000

In [67]:
import json

questions = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    splitted = data.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = [s.strip() for s in splitted]
    questions.extend(splitted)
    
questions = sorted(list(set(questions)))
len(questions)

4096

In [68]:
questions[:100]

['How many factors are there for 12? (1, 2, 3, 4, 6, 12)',
 'How many factors are there for 16? (1, 2, 4, 8, 16)',
 'How many factors does 12 have?',
 'How many factors of 12 are there?',
 'What is (-2)^4?',
 'What is (1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10)?',
 'What is (1 + 2 + 3 + 4 + 5)?',
 'What is (1 + 2 + 3 + 4) \\* 5?',
 'What is (1 + 2 + 3 \\* 4)?',
 'What is (1 + 2 + 3) \\* (1 + 2) \\* (1 + 3) ?',
 'What is (1 + 2 + 3) \\* 4?',
 'What is (1 + 2 × 3) × 4 - 5?',
 'What is (1 + 2) \\* (1 + 2) \\* (1 + 2)?',
 'What is (1 + 2) \\* (3 + 4)?',
 'What is (10 + 10 + 10 + 10) \\* 2.5?',
 'What is (10 + 10 + 10) \\* 3?',
 'What is (10 + 10) \\* (10 + 10) \\* 10?',
 'What is (10 + 10) \\* (10 + 10)?',
 'What is (10 + 10) \\* (10 - 5)?',
 'What is (10 + 10) \\* (2 + 2)?',
 'What is (10 + 10) \\* 10?',
 'What is (10 + 10) \\* 2?',
 'What is (10 + 15) \\* (10 - 5)?',
 'What is (10 + 2 \\* 5) - (3 + 4 \\* 2)?',
 'What is (10 + 2) / 3?',
 'What is (10 + 2) \\* (10 - 2) - (5 \\* 5)?',
 'What i

In [29]:
!mkdir answer-mixtral-simple-math-question

In [33]:
def answer(q, i):
    filename = f'answer-mixtral-simple-math-question/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [34]:
max_worker = 50

for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████████| 82/82 [14:34<00:00, 10.66s/it]


In [76]:
# data = []
# for no, q in enumerate(questions):
#     filename = f'answer-mixtral-simple-math-question/{no}.json'
#     if not os.path.exists(filename):
#         continue

#     with open(filename) as fopen:
#         a = json.load(fopen).strip()

#     d = {
#         'question': q,
#         'answer': a,
#     }
#     data.append(d)

In [77]:
data = []
for f in tqdm(glob('answer-mixtral-simple-math-question/*.json')):
    with open(f) as fopen:
        a = json.load(fopen).strip()
        if a.count('?') > 0:
            continue
        data.append(a)
        
data.extend(questions)
len(data)

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 127297.24it/s]


8154

In [78]:
data[:10]

['The order of operations, often remembered by the acronym PEMDAS, tells us to perform calculations in the following order: Parentheses, Exponents, Multiplication and Division (from left to right), and Addition and Subtraction (from left to right).\n\nSo for the expression 14 / 7 + 3:\n\nFirst, perform the division: 14 / 7 = 2\nThen, perform the addition: 2 + 3 = 5\n\nSo, 14 / 7 + 3 = 5.',
 'To find the product of 1111 multiplied by itself three times, you can do the multiplication step by step:\n\n1111 * 1111 = 1234321\n\nNow multiply the result by 1111:\n\n1234321 * 1111 = 1367643121\n\nSo, 1111 \\* 1111 \\* 1111 = 1367643121.',
 'The result of subtracting 9 from 69 is 60. You perform this operation by aligning the numbers by place value and subtracting the digit in the second number from the digit in the first number for each place value, starting from the rightmost place and moving left. In this case, you would subtract 9 (in the ones place) from 69, resulting in 60.',
 'To evaluat

In [79]:
with open('answer-mixtral-simple-math-question.texts', 'w') as fopen:
    for t in set(data):
        fopen.write(f'{json.dumps(t)}\n')

In [80]:
!wc -l answer-mixtral-simple-math-question.texts

8154 answer-mixtral-simple-math-question.texts


In [81]:
!cp answer-mixtral-simple-math-question.texts ../ctranslate2

In [90]:
mapping = {}
with open('../ctranslate2/answer-mixtral-simple-math-question.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']

In [91]:
with open('mixtral-simple-math.jsonl', 'w') as fopen_l:
    for no, q in enumerate(questions):
        filename = f'answer-mixtral-simple-math-question/{no}.json'
        if not os.path.exists(filename):
            continue

        with open(filename) as fopen:
            a = json.load(fopen).strip()
            
        if a.count('?') > 0:
            continue
            
        d = {
            'question': q,
            'question_ms': mapping.get(q),
            'answer': a,
            'answer_ms': mapping.get(a)
        }
        fopen_l.write(f'{json.dumps(d)}\n')

In [92]:
!head -n 3 mixtral-simple-math.jsonl

{"question": "How many factors are there for 12? (1, 2, 3, 4, 6, 12)", "question_ms": "Berapakah bilangan faktor untuk 12? (1, 2, 3, 4, 6, 12)", "answer": "Yes, you are correct. The factors of 12 are 1, 2, 3, 4, 6, and 12. A factor is a whole number that can be divided evenly into another number. In this case, each of these numbers divides evenly into 12 with no remainder.", "answer_ms": "Ya, anda betul. Faktor 12 ialah 1, 2, 3, 4, 6, dan 12. Faktor ialah nombor keseluruhan yang boleh dibahagikan sama rata kepada nombor lain. Dalam kes ini, setiap nombor ini dibahagikan sama rata kepada 12 tanpa baki."}
{"question": "How many factors are there for 16? (1, 2, 4, 8, 16)", "question_ms": "Berapakah bilangan faktor untuk 16? (1, 2, 4, 8, 16)", "answer": "Yes, you are correct. The factors of 16 are 1, 2, 4, 8, and 16. A factor is a whole number that can be divided evenly into another number. In this case, each of these numbers can be divided evenly into 16, so they are all factors of 16.", 

In [93]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-simple-math.jsonl',
    path_in_repo='mixtral-simple-math.jsonl',
    repo_id='mesolitica/mixtral-malaysian-general-qa',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/blob/main/mixtral-simple-math.jsonl'